<a href="https://colab.research.google.com/github/Heisnotanimposter/CustomGPT/blob/main/GPT_fine_tuning_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yaml
from glob import glob

#hh-rlhf predataset download

!curl -X "https://datasets-server.huggingface.co/splits?dataset=Anthropic%2Fhh-rlhf" > rlhf1.zip; unzip rlhf1.zip; rm rlhf1.zip
!curl -X "https://datasets-server.huggingface.co/splits?dataset=stanfordnlp%2FSHP"  > rlhf2.zip; unzip rlhf2.zip; rm rlhf2.zip

curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information
Archive:  rlhf1.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of rlhf1.zip or
        rlhf1.zip.zip, and cannot find rlhf1.zip.ZIP, period.
curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information
Archive:  rlhf2.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of rlhf2.zip or
        rlhf2.zip.zip, and cannot find rlhf2.zip.ZIP, period.


In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.7/686.7 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
#import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

In [ ]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default='./data_kochatgpt/kochatgpt_1_SFT.jsonl')
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default='./output_1_SFT')

args = parser.parse_args(args=[])

# test model
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 3

print(args)

Namespace(data_path_1_SFT='./data_kochatgpt/kochatgpt_1_SFT.jsonl', model_name='skt/kogpt2-base-v2', max_epochs=3, train_batch_size=8, output_dir='./output_1_SFT')


In [ ]:
## test & load skt gpt2 korean

import torch
from transformers import GPT2LMHeadModel

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
print(tokenizer.tokenize("Welcome to SKTGPT2 korean!"))


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁W', 'el', 'c', 'ome', '▁to', '▁K', 'O', 'G', 'P', 'T', '!']


In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '법원에서 김밥이랑 커피를 먹는 방법은?'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

법원에서 김밥이랑 커피를 먹는 방법은?
김밥을 먹고 나서 바로 밥과 함께 먹었어요.
그런데 이게 정말 맛있더라고요.
그리고 그 위에 #아몬드크림치즈 를 올려서 먹으면 더 맛있는 것 같아요!
저는 아빠가 좋아하는 치즈를 좋아하는데...
그래도 너무너무 잘 어울려서 좋았어요
맛있게 먹을 수 있는 메뉴들이 많아서 더욱 좋을 듯해요.
저희는 오늘 저녁은 꼭 먹어봐야겠다 싶어서 주문했어요.
오늘의 메인메인인 ``오리지널스프레이, 오렌지, 레드, 그린 등
